<a href="https://colab.research.google.com/github/Priyapanch/my_python_hub/blob/main/AWSS3StaticHosting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

UserId,NoteId,Notes
testuser,001,hello
testuser,002,this is my first note
student,001,DynamoDB is NoSQL
student,002,A DynamoDB table is schemaless
student,003,PartQL is a SQL compatible language for DynamoDB
student,004,I love DyDB
student,005,Maximum size of an item is ____ KB ?
newbie,001,Free swag code: 1234
newbie,002,I love DynamoDB

[S3]
bucket_name = notes-bucket-[initials]-[postalcode]
object_name = notes
source_file_extention = .csv
source_content_type = text/csv
processed_file_extension = .json
processed_content_type = application/json
metaData_key = myVal2
metaData_value = lab2-testing-upload

In [ ]:
#Convert csv to json

import boto3, botocore, json, csv, io, configparser

def main(s3Client):
    print('\nStart of convert object script\n')

    print('Reading configuration file for bucket name...')
    config = readConfig()
    bucket_name = config['bucket_name']
    source_file_name = config['object_name'] + config['source_file_extension']
    key_name = config['key_name']+ config['source_file_extension']
    processed_file_name = config['key_name'] + config['processed_file_extension']
    contentType = config['processed_content_type']
    metaData_key = config['metaData_key']
    metaData_value = config['metaData_value']

    #### Get the object from S3
    print('\nGetting the CSV object from S3 bucket')
    csvStr = getCSVFile(s3Client, bucket_name, key_name)

    ## Convert the object to the new format
    print('\nConverting CSV string to JSON...')
    jsonStr = convertToJSON(csvStr)

    ## Uploaded the converted object to S3
    print('Creating the new JSON object on S3')
    print(createObject(s3Client, bucket_name, processed_file_name, jsonStr, contentType, {metaData_key: metaData_value}))

    print('\nEnd of convert object script\n')

def getCSVFile(s3Client, bucket, key):
    bytes_buffer = io.BytesIO()

    ## Start TODO 6: Download the file contents to the
    ## bytes_buffer object so that it can be decoded to a string.

    s3Client.download_fileobj(
      Bucket=bucket,
      Key=key,
      Fileobj=bytes_buffer)

    ## End TODO 6

    byte_value = bytes_buffer.getvalue()
    return byte_value.decode('utf-8')

def createObject(s3Client, bucket, key, data, contentType, metadata={}):
    ## Start TODO 7: Create an S3 object with the converted data

    s3Client.put_object(
        Bucket=bucket,
        Key=key,
        Body=data,
        ContentType=contentType,
        Metadata=metadata
    )

    ## End TODO 7

    return 'Successfully Created Object\n'

def convertToJSON(input):
    jsonList = []
    keys = []

    csvReader = csv.reader(input.split('\n'), delimiter=",")

    for i, row in enumerate(csvReader):
        if i == 0:
            keys = row
        else:
            obj = {}
            for x, val in enumerate(keys):
                obj[val] = row[x]
            jsonList.append(obj)
    return json.dumps(jsonList, indent=4)

def readConfig():
    config = configparser.ConfigParser()
    config.read('./labRepo/config.ini')

    return config['S3']

# Create an S3 client to interact with the service and pass
# it to the main function that will create the buckets
client = boto3.client('s3')
try:
    main(client)
except botocore.exceptions.ClientError as err:
    print(err.response['Error']['Message'])
except botocore.exceptions.ParamValidationError as error:
    print(error)

In [ ]:
#create bucket
import boto3, botocore, configparser

def main(s3Client):
    print('\nStart of create bucket script\n')

    print('Reading configuration file for bucket name...')
    config = readConfig()
    bucket_name = config['bucket_name']

    print('Verifying that the bucket name is valid...')
    #### Verify that the bucket exists. The script with exit
    #### if the name is not valid for a new bucket.
    verifyBucketName(s3Client, bucket_name)
    print(bucket_name)

    #### Create the notes-bucket-
    createBucket(s3Client, bucket_name)

    #Pause until the the bucket is in the account
    print('\nConfirm that the bucket exists...')
    verifyBucket(s3Client, bucket_name)

    print('\nEnd of create bucket script\n')

def verifyBucketName(s3Client, bucket):
    try:
        ## Start TODO 2: enter a command that will check if a bucket already exists in AWS
        ## with the name built from your ini file input.
        s3Client.head_bucket(Bucket=bucket)
        ## End TODO 2

        # If the previous command is successful, the bucket is already in your account.
        raise SystemExit('This bucket has already been created in your account, exiting because there is nothing further to do!')
    except botocore.exceptions.ClientError as e:
        error_code = int(e.response['Error']['Code'])
        if error_code == 404:
          ## If you receive a 404 error code, a bucket with that name
          ##  does not exist anywhere in AWS.
          print('Existing Bucket Not Found, please proceed')
        if error_code == 403:
          ## If you receive a 403 error code, a bucket exists with that
          ## in another AWS account.
          raise SystemExit('This bucket has already owned by another AWS Account, change the suffix and try a new name!')

def createBucket(s3Client, name):
    session = boto3.session.Session()

    # Obtain the region from the boto3 session
    current_region = session.region_name
    print('\nCreating ' + name + ' in ' + current_region)

    # Start TODO 3: Create a new bucket in the users current region

    if current_region == 'us-east-1':
        s3Client.create_bucket(Bucket=name)
    else:
        s3Client.create_bucket(
          Bucket=name,
          CreateBucketConfiguration={
            'LocationConstraint': current_region
          })

    # End TODO 3:

    print('Success!')

def verifyBucket(s3Client, bucket):
    ## Start TODO 4: Complete the function so that it will
    ## pause and only proceed after the bucket exists.

    waiter = s3Client.get_waiter('bucket_exists')
    waiter.wait(Bucket=bucket)

    ## End TODO 4
    print('The bucket:' + bucket + ' is now available.')

def readConfig():
    config = configparser.ConfigParser()
    config.read('./labRepo/config.ini')

    return config['S3']

## TODO 1: Create an S3 client to interact with the service and pass
## it to the main function that will create the buckets

client = boto3.client('s3')

## End TODO 1

try:
    main(client)
except botocore.exceptions.ClientError as err:
    print(err.response['Error']['Message'])
except botocore.exceptions.ParamValidationError as error:
    print(error)

In [ ]:
#create object
import boto3, botocore, configparser

def main(s3Client):
    print('\nStart of create object script\n')
    ## Initialize variables for object creation

    print('Reading configuration file for bucket name...')
    config = readConfig()
    bucket_name = config['bucket_name']
    source_file_name = config["object_name"] + config['source_file_extension']
    key_name = config['key_name']+ config['source_file_extension']
    contentType = config['source_content_type']
    metaData_key = config['metaData_key']
    metaData_value = config['metaData_value']

    #### Create object in the s3 bucket
    print('Creating Object...')
    print(uploadObject(s3Client, bucket_name, source_file_name, key_name, contentType, {metaData_key: metaData_value}))

    print('\nEnd of create object script\n')

def uploadObject(s3Client, bucket, name, key, contentType, metadata={}):

    ## Start TODO 5: create a object by transferring the file to the S3 bucket,
    ## set the contentType of the file and add any metadata passed to this function.
    s3Client.upload_file(
        Bucket=bucket,
        Key=key,
        Filename=name,
        ExtraArgs={
            'ContentType': contentType,
            'Metadata': metadata
            }
    )
    ## End TODO 5
    return "Finished creating object\n"

def readConfig():
    config = configparser.ConfigParser()
    config.read('./labRepo/config.ini')

    return config['S3']

# Create an S3 client to interact with the service and pass
# it to the main function that will create the buckets
client = boto3.client('s3')
try:
    main(client)
except botocore.exceptions.ClientError as err:
    print(err.response['Error']['Message'])
except botocore.exceptions.ParamValidationError as error:
    print(error)

In [ ]:
#create s3 website
import boto3
import botocore
import json
import configparser


def main(s3Client):
    print('Starting create website function...\n')

    print('Reading configuration file for bucket name...')
    config = readConfig()
    bucket_name = config['bucket_name']

    # Upload html files
    print('Uploading files for the website...')
    uploadWebsiteFiles(s3Client, bucket_name)

    # Enable web hosting
    print('Enabling web hosting on the bucket...')
    enableWebHosting(s3Client, bucket_name)

    # Configure bucket policy
    print('Adding a bucket policy to allow traffic from the internet...')
    allowAccessFromWeb(s3Client, bucket_name)

    # Obtain the region from the boto3 session and print url
    session = boto3.session.Session()
    current_region = session.region_name
    print('\nYou can access the website at:\n')
    print('http://' + bucket_name + '.s3-website-' + current_region +'.amazonaws.com')

    print('\nEnd create website function...')


def uploadWebsiteFiles(s3Client, bucket):
    fileNames = getFileList()
    for obj in fileNames:
        key = obj['Name']
        filename = './labRepo/html/' + key
        contentType = obj['Content']
        # Start TODO 8: Upload html/index.html and html/error.html the the bucket
        s3Client.upload_file(
            Filename=filename,
            Bucket=bucket,
            Key=key,
            ExtraArgs={
                'ContentType': contentType
            }
        )
        # End TODO 8


def enableWebHosting(s3Client, bucket):
    # Start TODO 9: enable S3 web hosting using the objects you uploaded in the last method
    # as the index and error document for the website.
    s3Client.put_bucket_website(
        Bucket=bucket,
        WebsiteConfiguration={
            'ErrorDocument': {'Key': 'error.html'},
            'IndexDocument': {'Suffix': 'index.html'},
        }
    )
    # End TODO 9


def allowAccessFromWeb(s3Client, bucket):
    bucket_policy = {
        'Version': '2012-10-17',
        'Statement': [{
            'Effect': 'Allow',
            'Principal': '*',
            'Action': ['s3:GetObject'],
            'Resource': "arn:aws:s3:::" + bucket + '/*'
        }]
    }
    bucket_policy = json.dumps(bucket_policy)

    # Start TODO 10: Apply the provided bucket policy to the website bucket
    # to allow your objects to be accessed from the internet.
    s3Client.put_bucket_policy(
        Bucket=bucket,
        Policy=bucket_policy
    )
    # End TODO 10


def getFileList():
    return [
        {
            "Name": '404.png',
            "Content": 'image/png'
        },
        {
            "Name": 'header.png',
            "Content": 'image/png'
        },
        {
            "Name": 'error.html',
            "Content": 'text/html'
        },
        {
            "Name": 'index.html',
            "Content": 'text/html'
        },
        {
            "Name": 'styles.css',
            "Content": 'text/css'
        }
    ]


def readConfig():
    config = configparser.ConfigParser()
    config.read('./labRepo/config.ini')

    return config['S3']


# Create an S3 client to interact with the service and pass
# it to the main function that will create the buckets
client = boto3.client('s3')
try:
    main(client)
except botocore.exceptions.ClientError as err:
    print(err.response['Error']['Message'])
except botocore.exceptions.ParamValidationError as error:
    print(error)


{
  "Version": "2012-10-17",
  "Statement": [{
      "Effect": "Allow",
      "Principal": "*",
      "Action": ["s3:GetObject"],
      "Resource": "arn:aws:s3:::[BUCKET]/*"
    }]
}

{
  "IndexDocument": {
      "Suffix": "index.html"
  },
  "ErrorDocument": {
      "Key": "error.html"
  }
}